In [1]:
#All the imports & pyplot inline setting
import numpy as np
import pandas as pd
import random
from scipy.spatial import distance
from sklearn import preprocessing
import scipy.stats

In [9]:
n_file='project3_dataset2.txt'
#n_file='project3_dataset1.txt'
n_neighbours=3
n_folds = 10

In [10]:
df = pd.read_csv(n_file,header=None,delim_whitespace=True)

In [11]:
df = pd.read_csv(n_file,header=None,delim_whitespace=True)
shuffled_idx = np.random.permutation(df.index)
df = df.reindex(shuffled_idx)
data_truth = df.iloc[:,-1].values

In [12]:
def find_label():
    global data_probability
    global data_label
    data_label = np.argmax(data_probability,axis=1)

In [13]:
def calculate_probability(test,train,train_truth):
    global data_probability
    for i in range(df.shape[1]-1):
        if(df[i].dtype=='object'):
            le = preprocessing.LabelEncoder()
            le.fit(df[:][i])
            l_test = le.transform(test[:][i])
            l_test = l_test.reshape((l_test.shape[0], 1))
            l_train = le.transform(train[:][i])
            l_train = l_train.reshape((l_train.shape[0], 1))
            dict_0class = {}
            dict_1class = {}
            for i in range(len(le.classes_)):
                dict_0class[i] = (0.0 + (l_train[train_truth==0]==i).sum())/((train_truth==0).sum())
                dict_1class[i] = (0.0 + (l_train[train_truth==1]==i).sum())/((train_truth==1).sum())
            dm = np.zeros((test.shape[0],2))
            dm[:,0] = np.vectorize(dict_0class.get)(l_test).flatten()
            dm[:,1] = np.vectorize(dict_1class.get)(l_test).flatten()
        else:
            l_test = test[:][i]
            l_test = l_test.reshape((l_test.shape[0], 1))
            l_train = train[:][i]
            l_train = l_train.reshape((l_train.shape[0], 1))
            dm = np.zeros((test.shape[0],2))
            pdf_0 = scipy.stats.norm(l_train[train_truth==0].mean(), l_train[train_truth==0].std())
            pdf_1 = scipy.stats.norm(l_train[train_truth==1].mean(), l_train[train_truth==1].std())
            dm[:,0] = np.vectorize(pdf_0.pdf)(l_test).flatten()
            dm[:,1] = np.vectorize(pdf_1.pdf)(l_test).flatten()
        data_probability = np.multiply(data_probability,dm)

In [14]:
def calculate_measure(data_label,test_truth):
    total = data_label.shape[0]
    match = data_label[data_label.flatten()==test_truth.flatten()]
    non_match = data_label[data_label.flatten()!=test_truth.flatten()]
    pos = match[match==1].shape[0]
    neg = match[match==0].shape[0]
    false_pos = non_match[non_match==1].shape[0]
    false_neg = non_match[non_match==0].shape[0]
    cal_accuracy = (0.0 + pos + neg)/(0.0 + pos + neg + false_pos + false_neg)
    cal_precision = (0.0 + pos)/(0.0 + pos + neg)
    cal_recall = (0.0 + pos)/(0.0 + pos + false_neg)
    denom = (0.0 + cal_precision + cal_recall)
    if denom==0:
        denom=1
    cal_f_measure = (2.0*cal_precision*cal_recall)/denom
    return (cal_accuracy,cal_precision,cal_recall,cal_f_measure)

In [15]:
accuracy = 0.0
precision = 0.0
recall = 0.0
f_measure = 0.0
count=shuffled_idx.shape[0]//n_folds
for i in range(n_folds):
    test = df.iloc[count*i:count*(i+1)]
    train = pd.concat([df.iloc[0:count*i],df.iloc[count*(i+1):]])
    test_truth = data_truth[count*i:count*(i+1)]
    train_truth = pd.concat([df.iloc[:,-1].iloc[0:count*i],df.iloc[:,-1].iloc[count*(i+1):]]).values
    data_probability = np.ones((test.shape[0],2))
    data_label = np.zeros(test_truth.shape)
    calculate_probability(test,train,train_truth)
    find_label()
    current_accuracy,current_precision,current_recall,current_f_measure = calculate_measure(data_label,test_truth)
    print 'kFold ' + str(i) + ':\nAccuracy: ' + str(current_accuracy) + '\tPrecision: ' + str(current_precision) + \
    '\tRecall: ' + str(current_recall) + '\tF-measure: ' + str(current_f_measure)
    accuracy = accuracy + current_accuracy
    precision = precision + current_precision
    recall = recall + current_recall
    f_measure = f_measure + current_f_measure
print 'Average Accuracy: ' + str(accuracy/n_folds)
print 'Average Precision: ' + str(precision/n_folds)
print 'Average Recall: ' + str(recall/n_folds)
print 'Average F-Measure: ' + str(f_measure/n_folds)

kFold 0:
Accuracy: 0.673913043478	Precision: 0.41935483871	Recall: 0.764705882353	F-measure: 0.541666666667
kFold 1:
Accuracy: 0.760869565217	Precision: 0.371428571429	Recall: 0.65	F-measure: 0.472727272727
kFold 2:
Accuracy: 0.586956521739	Precision: 0.296296296296	Recall: 0.666666666667	F-measure: 0.410256410256
kFold 3:
Accuracy: 0.717391304348	Precision: 0.424242424242	Recall: 0.736842105263	F-measure: 0.538461538462
kFold 4:
Accuracy: 0.652173913043	Precision: 0.3	Recall: 0.642857142857	F-measure: 0.409090909091
kFold 5:
Accuracy: 0.673913043478	Precision: 0.322580645161	Recall: 0.555555555556	F-measure: 0.408163265306
kFold 6:
Accuracy: 0.673913043478	Precision: 0.451612903226	Recall: 0.875	F-measure: 0.595744680851
kFold 7:
Accuracy: 0.673913043478	Precision: 0.258064516129	Recall: 0.615384615385	F-measure: 0.363636363636
kFold 8:
Accuracy: 0.782608695652	Precision: 0.361111111111	Recall: 0.8125	F-measure: 0.5
kFold 9:
Accuracy: 0.760869565217	Precision: 0.228571428571	Recall: 0